In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpt
import os
import geopandas as gpd
from glob import glob
from gtfs_functions import Feed

## Variables:

In [ ]:
gtfs_path = "CT_GTFS.zip" #update this if needed
current_stops_data = "Stops Sorted.xlsx"
future_stops_data = "Stops Sorted Future Network.xlsx"
current_cov = 'PTN service coverage.csv'
future_cov = 'Future Transit Service Coverage.csv'
demand_2 = "TotalDemandIndexStandardized.xlsx"
comm_file = 'demand.csv'
census_file = "Census by Community 2019_20240401.geojson"
red_code = '201-20718'
blue_code = '202-20718'
orange_code = '303-20727'
yellow_code = '304-20727'
teal_code = '306-20727'
purple_code = '307-20727'
comm_list = []
red_list = []
blue_list = []
orange_list = []
yellow_list = []
purple_list = []
teal_list = []

# Functions Required

In [ ]:
def assign_grade(score):
    for grade, boundary in grade_boundaries.items():
        if score >= boundary:
            return grade
    return 'F'

In [ ]:
def assign_time(time_list):
    time_format_list = [t+':00' for t in time_list]
    return time_format_list

# Generate Dataset for SF and SC

## Input Required:

In [ ]:
t = input("Input time range and separate by comma (eg. 6am-9am: 6,7,8,9):")
t_l = t.split(',')
time_range_list = assign_time(t_l)
pattern = '|'.join(time_range_list)

## Generate Data Set Via GTFS:

In [ ]:
feed = Feed(gtfs_path, time_windows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
line_freq = feed.lines_freq
ptn = line_freq.loc[line_freq["route_name"].str.contains("Line|MAX")].sort_values(by=['window'])
input_hour_performance = ptn.loc[ptn['window'].str.contains(pattern)].sort_values(by=['route_name'])
n_trips = input_hour_performance.groupby('route_id')['ntrips'].sum()
mean_min_per_trip = input_hour_performance.groupby('route_id')['min_per_trip'].mean()
median_min_per_trip = input_hour_performance.groupby('route_id')['min_per_trip'].median()

# SF/SC Through Stops Sorted

## Input Required:

In [ ]:
while True:
    try:
        bus_capacity = int(input("Enter the capacity of busses (Only integer value accepted): "))
        train_capacity = int(input("Enter the capacity of trains (Only integer value accepted): "))
        break
    except ValueError:
        # If the input is not a valid integer, print an error message and continue the loop
        print("Please enter a valid integer value.")

## Analysis SF/SC:

In [ ]:
blue_trips = n_trips.loc[blue_code]
blue_freq = median_min_per_trip.loc[blue_code]
red_trips = n_trips.loc[red_code]
red_freq = median_min_per_trip.loc[red_code]
orange_trips = n_trips.loc[orange_code]
orange_freq = median_min_per_trip.loc[orange_code]
teal_trips = n_trips.loc[teal_code]
teal_freq = median_min_per_trip.loc[teal_code]
yellow_trips = n_trips.loc[yellow_code]
yellow_freq = median_min_per_trip.loc[yellow_code]
purple_trips = n_trips.loc[purple_code]
purple_freq = median_min_per_trip.loc[purple_code]

In [ ]:
comm_data = pd.read_csv(comm_file)
stop_df = pd.read_excel(current_stops_data, sheet_name = "Sheet2")
stop_df['Community'] = stop_df['Community'].str.rstrip('\n').str.replace('\n',',')

In [ ]:
for index, row in stop_df.iterrows():
    if 'Line' in row['Line']:
        if 'Red' in row['Line']:
            red_list.extend([com for com in row['Community'].split(',')])
        else:
            blue_list.extend([com for com in row['Community'].split(',')])
    else:
        if 'Orange' in row['Line']:
            orange_list.extend([com for com in row['Community'].split(',')])
        elif 'Purple' in row['Line']:
            purple_list.extend([com for com in row['Community'].split(',')])
        elif 'Teal' in row['Line']:
            teal_list.extend([com for com in row['Community'].split(',')])
        else:
            yellow_list.extend([com for com in row['Community'].split(',')])
            
comm_list = list(set(red_list+blue_list+orange_list+purple_list+teal_list+yellow_list))
clean_com_list = [item for item in comm_list if item.strip()]
service_df = pd.DataFrame(clean_com_list, columns=["Community"]).sort_values(by="Community")

service_df["Red Trips"] = service_df["Community"].isin(red_list) * red_trips
service_df["Red Freq"] = service_df["Community"].isin(red_list) * red_freq
service_df["Blue Trips"] = service_df["Community"].isin(blue_list) * blue_trips
service_df["Blue Freq"] = service_df["Community"].isin(blue_list) * blue_freq
service_df["Orange Trips"] = service_df["Community"].isin(orange_list) * orange_trips
service_df["Orange Freq"] = service_df["Community"].isin(orange_list) * orange_freq
service_df["Yellow Trips"] = service_df["Community"].isin(yellow_list) * yellow_trips
service_df["Yellow Freq"] = service_df["Community"].isin(yellow_list) * yellow_freq
service_df["Teal Trips"] = service_df["Community"].isin(teal_list) * teal_trips
service_df["Teal Freq"] = service_df["Community"].isin(teal_list) * teal_freq
service_df["Purple Trips"] = service_df["Community"].isin(purple_list) * purple_trips
service_df["Purple Freq"] = service_df["Community"].isin(purple_list) * purple_freq

# Merge the two DataFrames on the 'Community' and 'Community Name' columns
merged_data = pd.merge(service_df, comm_data, left_on='Community', right_on='Community Name', how='right')

# Update the 'Population' column in sup_fc with the corresponding values from comm_data
service_df['Population'] = service_df['Community'].apply(
    lambda x: merged_data.loc[merged_data['Community Name'] == x, 'Population in private households'].iloc[0]
    if x in merged_data['Community Name'].tolist() else 0
)

# dropping rows that have no population:
service_df = service_df[service_df['Population'] != 0]

# Calculate Index
service_df["Frequency Index"] = service_df.filter(like="Freq").sum(axis=1)
service_df['Total Trips'] = service_df.filter(regex="Red Trips|Blue Trips").sum(axis=1)*train_capacity + service_df.filter(regex="Orange Trips|Yellow Trips|Teal Trips|Yellow Trips|Purple Trips").sum(axis=1)*bus_capacity
service_df["Capacity Index"] = service_df['Total Trips']/service_df.filter(like='Population').sum(axis=1)

min_freq = min(service_df['Frequency Index'])
max_freq = max(service_df['Frequency Index'])
min_cap = min(service_df['Capacity Index'])
max_cap = max(service_df['Capacity Index'])

service_df['SS Frequency Index'] = (service_df['Frequency Index']-min_freq)/(max_freq-min_freq)
service_df['SS Capacity Index'] = (service_df['Capacity Index']-min_cap)/(max_cap-min_cap)

# SF SC SC

In [ ]:
curr_cov_file = pd.read_csv(current_cov)

supply_df_upper = service_df.copy()
supply_df_upper['Community'] = supply_df_upper['Community'].str.upper()

merged_data = pd.merge(supply_df_upper, curr_cov_file, how='right', left_on='Community', right_on='NAME')

final_supply = merged_data[['NAME','SS Frequency Index','SS Capacity Index','SS_Ser_Cov']]

final_supply = final_supply.rename(columns={'SS_Ser_Cov':"SS Coverage Index"})
final_supply = final_supply.fillna(0)

final_supply['Average Supply Index'] = final_supply[['SS Frequency Index','SS Capacity Index','SS Coverage Index']].mean(axis=1)

# Merge Demand and Supply

In [ ]:
demand_df = pd.read_excel(demand_2) # This will be data that has demand index, so if the data is already in a data frame, use that
demand_df['NAME'] = demand_df['NAME'].str.upper()

dns_merge = pd.merge(final_supply,demand_df, how='right', left_on='NAME', right_on='NAME')

final_current_df = dns_merge[['NAME',"Low Income Index","Seniors Index","Rent Index","Total Community Index", "SS Frequency Index", "SS Capacity Index",'SS Coverage Index',"Standardized Total Community Index","Average Supply Index"]]
final_current_df = final_current_df.rename(columns={'Standardized Total Community Index': 'Community Demand Index'})
final_current_df = final_current_df.fillna(0)

final_current_df["Transit Gap"] = final_current_df['Community Demand Index'] - final_current_df['Average Supply Index']

percentiles = final_current_df['Transit Gap'].quantile([0.2,0.4,0.6,0.8])

grade_boundaries = { # Create percentiles to assign proper letter grade
    'A': percentiles[0.8],  # Top 20% get an 'A'
    'B': percentiles[0.6],  # Next 20% get a 'B'
    'C': percentiles[0.4],  # Next 20% get a 'C'
    'D': percentiles[0.2],  # Next 20% get a 'D'
    'F': 0                   # Bottom 20% get an 'F'
}
final_current_df['Letter Grade'] = final_current_df['Transit Gap'].apply(assign_grade)



# Boundaries + Final Data Set

In [ ]:
comm_geo = gpd.read_file(census_file)
dns_bound_data = pd.merge(final_current_df, comm_geo, how='left', left_on='NAME', right_on='name')
current_dns = gpd.GeoDataFrame(dns_bound_data[['NAME',"Low Income Index","Seniors Index","Rent Index","Total Community Index", "SS Frequency Index", "SS Capacity Index",'SS Coverage Index',"Community Demand Index","Average Supply Index",'Transit Gap','Letter Grade','geometry']])
